In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
import numpy as np
from math import sqrt

In [55]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE

In [56]:
cdata = pd.read_csv("london_flows.csv")

In [57]:
cdata.head(5)

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,1,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167


In [58]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

#take the variables and produce logarithms of them
x_variables = ["population", "jobs", "distance"]
log_x_vars = []
for x in x_variables:
    cdata[f"log_{x}"] = np.log(cdata[x])
    log_x_vars.append(f"log_{x}")



In [59]:
#create the formula
formula = 'flows ~ log_population + log_jobs + log_distance'

#run the regression
uncosim = smf.glm(formula = formula, 
                  data=cdata, 
                  family=sm.families.Poisson()).fit()
#extract the summary of the constrained model
print(uncosim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61456
Model:                            GLM   Df Residuals:                    61452
Model Family:                 Poisson   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.2596e+06
Date:                Sun, 07 May 2023   Deviance:                   2.3119e+06
Time:                        16:27:56   Pearson chi2:                 4.62e+06
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -3.5630      0.014   -261.

In [6]:
cdatasub = cdata.copy()
#first assign the parameter values from the model to the appropriate variables
K = uncosim.params[0]
alpha = uncosim.params[1]
gamma = uncosim.params[2]
beta = -uncosim.params[3]

#now plug everything back into the Equation 6 model ... 
#be careful with the negative signing of the parameter beta
cdatasub["unconstrainedEst2"] = np.exp(K 
                                       + alpha*cdatasub["log_population"] 
                                       + gamma*cdatasub["log_jobs"] 
                                       - beta*cdatasub["log_distance"])

#or we can just extract the results from the actual poisson regression and apply them to the data
predictions = uncosim.get_prediction()
predictions_summary_frame = predictions.summary_frame()
cdatasub["fitted"] = predictions_summary_frame["mean"]

In [7]:
#round the numbers so that we don't get a half of a person
cdatasub["unconstrainedEst2"] = round(cdatasub["unconstrainedEst2"], 0)
#convert to integers
cdatasub["unconstrainedEst2"] = cdatasub["unconstrainedEst2"].astype(int)
#check that the sum of these estimates make sense
sum(cdatasub["unconstrainedEst2"])

1559865

In [50]:
CalcRSquared(cdatasub["flows"], cdatasub["unconstrainedEst2"])

0.3196566214242869

In [52]:
CalcRMSE(cdatasub["flows"], cdatasub["unconstrainedEst2"])

108.404

## production constrained

In [60]:
#create the formula (the "-1" indicates no intercept in the regression model).
formula = 'flows ~ station_origin + log_jobs + log_distance-1'
#run a production constrained sim
prodSim = smf.glm(formula = formula, data=cdata, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
#print(prodSim.summary())

In [9]:
cdatasub =  cdata.copy()
#We can do this by pulling out the parameter values
coefs = pd.DataFrame(prodSim.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(station_origin)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "")
#then once you have done this you can join them back into the dataframes
cdatasub = cdatasub.merge(coefs, left_on="station_origin", right_on="coef", how = "left")
cdatasub.drop(columns = ["coef"], inplace = True)
#check this has worked
cdatasub.head()

/tmp/ipykernel_117/2519043002.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  coefs["coef"] = coefs["coef"].str.replace(x, "")


,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,alpha_i
0,Abbey Road,Bank and Monument,1,599,78549,8131.525097,6.395262,11.271478,9.003504,3.257207
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,3.257207
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,3.257207
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,3.257207
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,3.257207


In [10]:
alpha_i = prodSim.params[0:399]
gamma = prodSim.params[399]
beta = -prodSim.params[400]

In [11]:
print(beta)

0.8569660931816073


In [78]:
cdatasub["prodsimest1"] = np.exp(cdatasub["alpha_i"]+gamma*cdatasub["log_jobs"] 
                                 - beta*cdatasub["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
cdatasub.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,alpha_i,prodsimest1
0,Abbey Road,Bank and Monument,1,599,78549,8131.525097,6.395262,11.271478,9.003504,3.257207,55.734042
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,3.257207,1.088455
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,3.257207,2.969901
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,3.257207,66.983301
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,3.257207,49.668463
5,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842,3.257207,2.851270
6,Abbey Road,Custom House,1,599,845,3824.855630,6.395262,6.739337,8.249276,3.257207,3.516937
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909,6.395262,7.466228,9.048280,3.257207,3.063880
8,Abbey Road,Cyprus,7,599,850,6532.099618,6.395262,6.745236,8.784484,3.257207,2.233062
9,Abbey Road,Devons Road,1,599,611,3958.324171,6.395262,6.415097,8.283576,3.257207,2.675895


In [79]:
CalcRSquared(cdatasub["flows"], cdatasub["prodsimest1"])

0.3894872603740291

In [83]:
CalcRMSE(cdatasub["flows"], cdatasub["prodsimest1"])

102.666

## attraction

In [74]:
#create the formula (the "-1" indicates no intercept in the regression model).
attr_form = 'flows ~ station_destination + log_population + log_distance-1'
#run a destination constrained sim
attrSim = smf.glm(formula = attr_form, data=cdatasub, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
#print(attrSim.summary())

In [81]:
#get the predictions
predictions = attrSim.get_prediction(cdatasub[["station_destination", "log_population", "log_distance"]])
predictions_summary_frame = predictions.summary_frame()
cdatasub["attrsimFitted"] = round(predictions_summary_frame["mean"],0)



In [73]:
CalcRSquared(cdatasub["flows"], cdatasub["attrsimFitted"])

0.3489435512636817

In [82]:
CalcRMSE(cdatasub["flows"], cdatasub["attrsimFitted"])

106.009

## double constrained

In [12]:
#create the formula (the "-1" indicates no intercept in the regression model).
dbl_form = 'flows ~ station_destination + station_origin + log_distance -1'
#run a doubly constrained sim
doubSim = smf.glm(formula = dbl_form, data=cdatasub, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
#print(doubSim.summary())

In [13]:
#get the estimates
cdatasub["doubsimfitted"] = np.round(doubSim.mu)

In [90]:
CalcRSquared(cdatasub["flows"],cdatasub["doubsimfitted"])

0.410069803247956

In [91]:
CalcRMSE(cdatasub["flows"],cdatasub["doubsimfitted"])

100.989

## entropy

In [14]:
#create some Oi and Dj columns in the dataframe and store row and column totals in them:
#to create O_i, take cdatasub ...then... group by origcodenew ...then... summarise by calculating the sum of Total
O_i = pd.DataFrame(cdatasub.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
cdatasub = cdatasub.merge(O_i, on = "station_origin", how = "left" )

D_j = pd.DataFrame(cdatasub.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
cdatasub = cdatasub.merge(D_j, on = "station_destination", how = "left" )

In [15]:

# Here is the entropy maximising approach for a known beta.
# Plug in the required values in this function to solve.

def balance_doubly_constrained(pd, orig_field, dest_field, Oi_field, Dj_field, cij_field, beta, 
                               cost_function, Ai_name = "Ai_new", Bj_name = "Bj_new", converge=0.001):
    # Define some variables
    Oi = pd[[orig_field, Oi_field]]
    Dj = pd[[dest_field,Dj_field]]    
    if cost_function.lower() in ['power','pow']:
        beta_cij = np.exp(beta * np.log(pd[cij_field]))
    elif cost_function.lower() in ['exponential','exp']:
        beta_cij = np.exp(beta * pd[cij_field])
    else:
        return "Cost function not specified properly, use 'exp' or 'pow'"
    
    # Create some helper variables
    cnvg = 1
    iteration = 0
    # Now iteratively rebalance the Ai and Bj terms until convergence
    while cnvg > converge:
        if iteration == 0:
            # This first condition sets starting values for Ai and Bj
            # NB sets starting value of Ai assuming Bj is a vector of 1s.
            # We've already established beta_cij with the appropriate cost function, so...
            Oi = Oi.assign(Ai = Dj[Dj_field] * beta_cij)
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Now set up Bjs using starting values of Ai
            Dj = Dj.assign(Bj = Oi['Ai_old'] * Oi[Oi_field] * beta_cij)
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Increment loop
            iteration += 1
        else:
            # This bit is the iterated bit of the loop which refines the values of Ai and Bj
            # First Ai
            Oi['Ai'] = Dj['Bj_old'] * Dj[Dj_field] * beta_cij
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Drop temporary Ai
            Oi.drop('Ai', axis=1, inplace=True)
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True)
            # Calculate the difference between old and new Ais
            Oi['diff'] = np.absolute((Oi['Ai_old'] - Oi['Ai'])/Oi['Ai_old'])
            # Set new Ais to Ai_old
            Oi['Ai_old'] = Oi['Ai']
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Then Bj
            Dj['Bj'] = Oi['Ai_old'] * Oi[Oi_field] * beta_cij
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Drop temporary Bj
            Dj.drop('Bj', axis=1, inplace=True)
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True)
            # Calculate the difference between old and new Bjs
            Dj['diff'] = np.absolute((Dj['Bj_old'] - Dj['Bj'])/Dj['Bj_old'])
            # Set new Bjs to Bj_old
            Dj['Bj_old'] = Dj['Bj']
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Assign higher sum difference from Ai or Bj to cnvg
            cnvg = np.maximum(Oi['diff'].sum(),Dj['diff'].sum())
            
            # Print and increment loop
            print("Iteration:", iteration)
            iteration += 1

    # When the while loop finishes add the computed Ai_old and Bj_old to the dataframe and return
    pd[Ai_name] = Oi['Ai_old']
    pd[Bj_name] = Dj['Bj_old']
    return pd

In [22]:
# Use the beta we got from the inverse power model
beta = -doubSim.params[-1]
# Get the balancing factors.
cdatasub = balance_doubly_constrained(cdatasub,'station_origin','station_destination','O_i','D_j','distance',-beta,'power')

# Now predict the model again using the new Ai and Dj fields.
cdatasub['SIM_est_pow'] = np.round(cdatasub['O_i'] * cdatasub['Ai_new'] * cdatasub['D_j'] * cdatasub['Bj_new'] * 
                                   np.exp(np.log(cdatasub['distance'])*-beta))
# Check out the matrix
pd.pivot_table(cdatasub,values='SIM_est_pow',index ='station_origin',columns='station_destination',fill_value=0,aggfunc=sum,margins=True)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22


station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,27,603.0
Acton Central,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,1247.0
Acton Town,0,0,0,16,15,0,12,2,0,17,...,30,4,5,12,0,2,0,3,0,3797.0
Aldgate,0,0,2,0,43,0,0,1,0,20,...,8,0,3,2,0,1,0,1,0,3006.0
Aldgate East,0,0,2,49,0,0,1,1,0,21,...,9,1,3,2,0,1,0,1,0,3258.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0,0,7,31,33,0,0,0,0,28,...,29,0,10,0,0,0,0,0,0,4897.0
Woodgrange Park,0,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,542.0
Woodside Park,0,0,6,18,17,0,3,0,0,22,...,21,0,6,0,0,0,0,0,0,3121.0


In [20]:
print(beta)

0.8924008290511345


In [17]:
cdatasub.head(5)

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,alpha_i,doubsimfitted,O_i,D_j,Ai_new,Bj_new,SIM_est_pow
0,Abbey Road,Bank and Monument,1,599,78549,8131.525097,6.395262,11.271478,9.003504,3.257207,56.0,603,78555,0.005067,0.719333,56.0
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,3.257207,2.0,603,453,0.005067,4.997638,2.0
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,3.257207,3.0,603,670,0.005067,2.111967,3.0
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,3.257207,77.0,603,58781,0.005067,0.874895,77.0
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,3.257207,45.0,603,15465,0.005067,0.934289,45.0


In [23]:
CalcRSquared(cdatasub["flows"], cdatasub['SIM_est_pow'])

0.4100698032479661

In [24]:
CalcRMSE(cdatasub["flows"], cdatasub['SIM_est_pow'])

100.989

## non-log

In [35]:
# Run a doubly constrained SIM with a negative exponential cost function.
doubsim_form = "flows ~ station_origin + station_destination + distance -1"
doubsim1 = smf.glm(formula=doubsim_form, data = cdatasub, family = sm.families.Poisson()).fit()
#print(doubsim1.summary())

In [36]:
cdatasub["doubsimfitted1"] = np.round(doubsim1.mu,0)

In [52]:
# Use the beta we got from the negative exponential model
beta = -doubsim1.params[-1]
#beta = -0.00014
# Get the balancing factors. NB Setting of new field names for Ai and Bj.
cdatasub = balance_doubly_constrained(cdatasub,'station_origin','station_destination','O_i','D_j','distance',-beta,'exponential','Ai_exp','Bj_exp')

# Now predict the model again using the new Ai and Dj fields.
cdatasub['SIM_est_exp'] = np.round(cdatasub['O_i'] * cdatasub['Ai_exp'] * cdatasub['D_j'] * cdatasub['Bj_exp'] * 
                                   np.exp(cdatasub['distance']*-beta))
# Check out the matrix
pd.pivot_table(cdatasub,values='SIM_est_exp',index ='station_origin',columns='station_destination',fill_value=0,aggfunc=sum,margins=True)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23


station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,31,603.0
Acton Central,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1252.0
Acton Town,0,0,0,11,10,0,18,0,0,13,...,40,4,2,18,0,0,0,1,0,3792.0
Aldgate,0,0,2,0,32,0,0,0,0,24,...,8,0,3,2,0,1,0,1,0,2994.0
Aldgate East,0,0,2,37,0,0,0,0,0,25,...,8,1,4,2,0,1,0,1,0,3256.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0,0,2,37,43,0,0,0,0,25,...,8,0,7,0,0,0,0,0,0,4894.0
Woodgrange Park,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,541.0
Woodside Park,0,0,2,17,16,0,1,0,0,25,...,10,0,5,0,0,0,0,0,0,3120.0


In [53]:
CalcRSquared(cdatasub["flows"], cdatasub['SIM_est_exp'])

0.4977804261817628

In [39]:
print(beta)

0.0001469789967357215


In [105]:
CalcRMSE(cdatasub["flows"], cdatasub['SIM_est_exp'])

93.446